In [1]:
import torch
from transformers import GPTQConfig, AutoTokenizer, AutoModelForSequenceClassification,AutoModelForCausalLM,Gemma2ForCausalLM, BitsAndBytesConfig
import logging
import pandas as pd
import pickle
from datasets import load_from_disk
import os
os.chdir("/group-volume/binfeng/wsdm/stage_qft")
from utils import *


/home/user/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
MODEL_PATH = "/group-volume/binfeng/wsdm/ckpt/phi4_4k_dare"
MAX_LEN = 4000
MAX_PROMPT_LENGTH = 800

In [5]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

In [6]:
# data = pd.read_parquet("/group-volume/binfeng/wsdm/stage_merge/qwen_ft_text.parquet").sample(n=2, random_state=2023)
# data = data["text"].to_list()
# gptq_config = GPTQConfig(bits=4, dataset = data, model_seqlen=3000, group_size = 128,
#                          exllama_config={"version":2},
#                          tokenizer=tokenizer)

bnb_config =  BitsAndBytesConfig(
        load_in_4bit = True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
    )

# bnb_config =  BitsAndBytesConfig(
#         load_in_8bit = True
#     )

merged_quantized_model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_PATH,
    # torch_dtype=torch.bfloat16,
    num_labels=2,
    device_map='auto',
    quantization_config=bnb_config,
    )

Loading checkpoint shards: 100%|██████████| 6/6 [02:40<00:00, 26.74s/it]


In [7]:
merged_quantized_model.save_pretrained("/group-volume/binfeng/wsdm/ckpt/q4/phi4_4k_dare_bnb_q4")
tokenizer.save_pretrained("/group-volume/binfeng/wsdm/ckpt/q4/phi4_4k_dare_bnb_q4")

('/group-volume/binfeng/wsdm/ckpt/q4/phi4_4k_dare_bnb_q4/tokenizer_config.json',
 '/group-volume/binfeng/wsdm/ckpt/q4/phi4_4k_dare_bnb_q4/special_tokens_map.json',
 '/group-volume/binfeng/wsdm/ckpt/q4/phi4_4k_dare_bnb_q4/vocab.json',
 '/group-volume/binfeng/wsdm/ckpt/q4/phi4_4k_dare_bnb_q4/merges.txt',
 '/group-volume/binfeng/wsdm/ckpt/q4/phi4_4k_dare_bnb_q4/added_tokens.json',
 '/group-volume/binfeng/wsdm/ckpt/q4/phi4_4k_dare_bnb_q4/tokenizer.json')